In [1]:
import pandas as pd
import yfinance as yf
import json
from tqdm import tqdm
import concurrent.futures
from urllib.parse import urlparse
import bisect
import time
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter

class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

HOUR = 60 * 60
YF_HISTORY_LIMIT = HOUR * 24 * 730      # limit by the yfinance package

yf_news_data_file_path = 'YahooFinanceFiltered-TickerTick-stock-news-dataset.2023-11-23.json'
with open(yf_news_data_file_path, "r") as f:
    yf_news_data = json.load(f)

sp500_data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500_tickers = set(sp500_data['Symbol'].tolist())

sp100_data = pd.read_html('https://en.wikipedia.org/wiki/S%26P_100')[2]
sp100_tickers = set(sp100_data['Symbol'].tolist())

sp500_tickers.remove('BRK.B')
sp100_tickers.remove('BRK.B')

sp500_tickers

{'A',
 'AAL',
 'AAPL',
 'ABBV',
 'ABNB',
 'ABT',
 'ACGL',
 'ACN',
 'ADBE',
 'ADI',
 'ADM',
 'ADP',
 'ADSK',
 'AEE',
 'AEP',
 'AES',
 'AFL',
 'AIG',
 'AIZ',
 'AJG',
 'AKAM',
 'ALB',
 'ALGN',
 'ALL',
 'ALLE',
 'AMAT',
 'AMCR',
 'AMD',
 'AME',
 'AMGN',
 'AMP',
 'AMT',
 'AMZN',
 'ANET',
 'ANSS',
 'AON',
 'AOS',
 'APA',
 'APD',
 'APH',
 'APTV',
 'ARE',
 'ATO',
 'AVB',
 'AVGO',
 'AVY',
 'AWK',
 'AXON',
 'AXP',
 'AZO',
 'BA',
 'BAC',
 'BALL',
 'BAX',
 'BBWI',
 'BBY',
 'BDX',
 'BEN',
 'BF.B',
 'BG',
 'BIIB',
 'BIO',
 'BK',
 'BKNG',
 'BKR',
 'BLDR',
 'BLK',
 'BMY',
 'BR',
 'BRO',
 'BSX',
 'BWA',
 'BX',
 'BXP',
 'C',
 'CAG',
 'CAH',
 'CARR',
 'CAT',
 'CB',
 'CBOE',
 'CBRE',
 'CCI',
 'CCL',
 'CDNS',
 'CDW',
 'CE',
 'CEG',
 'CF',
 'CFG',
 'CHD',
 'CHRW',
 'CHTR',
 'CI',
 'CINF',
 'CL',
 'CLX',
 'CMA',
 'CMCSA',
 'CME',
 'CMG',
 'CMI',
 'CMS',
 'CNC',
 'CNP',
 'COF',
 'COO',
 'COP',
 'COR',
 'COST',
 'CPAY',
 'CPB',
 'CPRT',
 'CPT',
 'CRL',
 'CRM',
 'CSCO',
 'CSGP',
 'CSX',
 'CTAS',
 'CTLT',
 'CTRA

In [2]:
session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND*5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)
# session = requests_cache.CachedSession('yfinance.cache')
session.headers['User-agent'] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
session

<CachedSession(cache=<SQLiteCache(name=http_cache)>, settings=CacheSettings(expire_after=-1))>

In [3]:
query_startime = int(time.time()) - (YF_HISTORY_LIMIT - 24 * HOUR)

for story in yf_news_data:
    if "unix_timestamp" in story and type(story["unix_timestamp"]) == int:
        query_endtime = story["unix_timestamp"] + 5 * 24 * HOUR
        break
else:
    raise Exception("Data appears to be empty or missing unix_timestamp field.")

print(f"Data query range: ({pd.Timestamp(query_startime, unit='s', tz='US/Eastern')}, {pd.Timestamp(query_endtime, unit='s', tz='US/Eastern')})")

ticker_history = {}
ticker_info = {}

for ticker in tqdm(sp100_tickers):
    for retry in range(5):
      try:
          yfTicker = yf.Ticker(ticker, session=session)
          ticker_info[ticker] = yfTicker.info
          hist_df = yfTicker.history(start=query_startime, end=query_endtime, interval='1h', raise_errors=True)
          hist_df['ts_int'] = hist_df.index.values.astype('int64') // 1e9
          ticker_history[ticker] = hist_df
          break
      except Exception as e:
          print(f"Error fetching {ticker}: {e}")
          time.sleep(2)
    else:
      print(f"Skipping {ticker}")

len(ticker_history), len(ticker_info)

Data query range: (2022-05-02 01:37:29-04:00, 2023-11-29 01:45:18-05:00)


  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [04:05<00:00,  2.46s/it]


(100, 100)

In [4]:
columns = ["datetime", "title", "description", "ticker", "company", "sector", "industry", "change_pct"]
column_idx = {col: i for i, col in enumerate(columns)}

def process_valid_story(story, tickers):
    rows = []

    timestamp_int = story["unix_timestamp"]
    timestamp = pd.Timestamp(timestamp_int, unit='s', tz='US/Eastern')

    row = [None] * len(columns)
    row[column_idx["datetime"]] = timestamp
    row[column_idx["title"]] = story["title"]
    row[column_idx["description"]] = story["description"] if "description" in story else ""

    for ticker in tickers:
        init_ts_int = timestamp_int - 2 * HOUR
        final_ts_int = timestamp_int + 6 * HOUR

        hist_df = ticker_history[ticker]
        info = ticker_info[ticker]

        ts_ints = hist_df["ts_int"].values

        init_idx = bisect.bisect_right(ts_ints, init_ts_int) - 1                 # bisect_right - 1: rightmost value less than or equal to x
        final_idx = bisect.bisect_left(ts_ints, final_ts_int, lo=init_idx+1)     # bisect_left: leftmost item greater than or equal to x

        if init_idx == 0 and ts_ints[init_idx] > init_ts_int:
            print("aaaaa")
            continue

        init_price = hist_df.iloc[init_idx]["Close"]
        final_price = hist_df.iloc[final_idx]["Close"]
        change_pct = (final_price - init_price) / init_price * 100
        
        row_copy = row.copy()
        row_copy[column_idx["ticker"]] = ticker
        row_copy[column_idx["company"]] = info["longName"]
        row_copy[column_idx["sector"]] = info["sector"]
        row_copy[column_idx["industry"]] = info["industry"]
        row_copy[column_idx["change_pct"]] = change_pct
        rows.append(row_copy)

    return rows

In [5]:
dataset = []
errors = []
future_story_map = {}
now = time.time()

with concurrent.futures.ThreadPoolExecutor() as executor:
    with open(yf_news_data_file_path, "r") as f:
        for story in json.load(f):
            tickers = [T for t in story.get("tickers_direct", []) + story.get("tickers_indirect", []) if (T := t.upper()) in sp100_tickers]
            if len(tickers) == 0:
                continue

            if "unix_timestamp" not in story and type(story["unix_timestamp"]) != int:
                continue

            if now - story["unix_timestamp"] >= YF_HISTORY_LIMIT - 24 * HOUR:
                break

            if "description" in story and len(story["description"]) > 0:
                future_story_map[executor.submit(process_valid_story, story, tickers)] = story
    
len(future_story_map)

77440

In [6]:
for future in tqdm(concurrent.futures.as_completed(future_story_map.keys()), total=len(future_story_map)):
    try:
        rows = future.result()
        dataset.extend(rows)
            
    except Exception as e:
        story = future_story_map[future]
        if "id" in story:
            errors.append((story["id"], e))

print(f"{len(errors)} errors: {errors}")

df = pd.DataFrame(dataset, columns=columns)
df.sort_values(by="datetime", inplace=True, ascending=False)
df

100%|██████████| 77440/77440 [00:00<00:00, 274334.32it/s]


0 errors


,datetime,title,description,ticker,company,sector,industry,change_pct
32297,2023-11-24 00:00:06-05:00,OpenAI turmoil exposes threat to Microsoft’s i...,Microsoft chief executive Satya Nadella’s deci...,MSFT,Microsoft Corporation,Technology,Software—Infrastructure,-0.500163
44106,2023-11-23 19:00:31-05:00,10 Can’t Miss Black Friday Electronics Deals a...,The biggest shopping day of the season is upon...,COST,Costco Wholesale Corporation,Consumer Defensive,Discount Stores,0.592448
83805,2023-11-23 17:43:45-05:00,UPDATE 1-German union Verdi calls for strikes ...,German trade union Verdi has called on members...,AMZN,"Amazon.com, Inc.",Consumer Cyclical,Internet Retail,-0.825589
739,2023-11-23 16:47:00-05:00,Corrections & Amplifications - The success of ...,The success of blood thinners being developed ...,BMY,Bristol-Myers Squibb Company,Healthcare,Drug Manufacturers - General,0.323559
21527,2023-11-23 15:25:11-05:00,EU mulls wider scope for cybersecurity certifi...,The European Union is considering broadening t...,GOOGL,Alphabet Inc.,Communication Services,Internet Content & Information,-1.429600
...,...,...,...,...,...,...,...,...
17386,2022-05-02 04:56:08-04:00,Zacks Investment Ideas feature highlights: Alp...,"Alphabet, Amazon and Tesla have been highlight...",GOOGL,Alphabet Inc.,Communication Services,Internet Content & Information,-16.099594
17384,2022-05-02 04:56:08-04:00,Zacks Investment Ideas feature highlights: Alp...,"Alphabet, Amazon and Tesla have been highlight...",TSLA,"Tesla, Inc.",Consumer Cyclical,Auto Manufacturers,18.298315
17383,2022-05-02 04:56:08-04:00,Zacks Investment Ideas feature highlights: Alp...,"Alphabet, Amazon and Tesla have been highlight...",AMZN,"Amazon.com, Inc.",Consumer Cyclical,Internet Retail,-17.533490
17385,2022-05-02 04:56:08-04:00,Zacks Investment Ideas feature highlights: Alp...,"Alphabet, Amazon and Tesla have been highlight...",GOOG,Alphabet Inc.,Communication Services,Internet Content & Information,-16.361722


In [7]:
df['datetime'] = df['datetime'].dt.strftime('%a %d %b %Y, %I:%M%p')
df

,datetime,title,description,ticker,company,sector,industry,change_pct
32297,"Fri 24 Nov 2023, 12:00AM",OpenAI turmoil exposes threat to Microsoft’s i...,Microsoft chief executive Satya Nadella’s deci...,MSFT,Microsoft Corporation,Technology,Software—Infrastructure,-0.500163
44106,"Thu 23 Nov 2023, 07:00PM",10 Can’t Miss Black Friday Electronics Deals a...,The biggest shopping day of the season is upon...,COST,Costco Wholesale Corporation,Consumer Defensive,Discount Stores,0.592448
83805,"Thu 23 Nov 2023, 05:43PM",UPDATE 1-German union Verdi calls for strikes ...,German trade union Verdi has called on members...,AMZN,"Amazon.com, Inc.",Consumer Cyclical,Internet Retail,-0.825589
739,"Thu 23 Nov 2023, 04:47PM",Corrections & Amplifications - The success of ...,The success of blood thinners being developed ...,BMY,Bristol-Myers Squibb Company,Healthcare,Drug Manufacturers - General,0.323559
21527,"Thu 23 Nov 2023, 03:25PM",EU mulls wider scope for cybersecurity certifi...,The European Union is considering broadening t...,GOOGL,Alphabet Inc.,Communication Services,Internet Content & Information,-1.429600
...,...,...,...,...,...,...,...,...
17386,"Mon 02 May 2022, 04:56AM",Zacks Investment Ideas feature highlights: Alp...,"Alphabet, Amazon and Tesla have been highlight...",GOOGL,Alphabet Inc.,Communication Services,Internet Content & Information,-16.099594
17384,"Mon 02 May 2022, 04:56AM",Zacks Investment Ideas feature highlights: Alp...,"Alphabet, Amazon and Tesla have been highlight...",TSLA,"Tesla, Inc.",Consumer Cyclical,Auto Manufacturers,18.298315
17383,"Mon 02 May 2022, 04:56AM",Zacks Investment Ideas feature highlights: Alp...,"Alphabet, Amazon and Tesla have been highlight...",AMZN,"Amazon.com, Inc.",Consumer Cyclical,Internet Retail,-17.533490
17385,"Mon 02 May 2022, 04:56AM",Zacks Investment Ideas feature highlights: Alp...,"Alphabet, Amazon and Tesla have been highlight...",GOOG,Alphabet Inc.,Communication Services,Internet Content & Information,-16.361722


In [8]:
df.to_csv("dataset.csv", index=False)

In [9]:
# https://stackoverflow.com/questions/5543651/computing-standard-deviation-in-a-stream
class OnlineStats:
    def __init__(self, iterable=None, ddof=1):
        self.ddof, self.n, self.mean, self.M2 = ddof, 0, 0.0, 0.0
        if iterable is not None:
            for datum in iterable:
                self.include(datum)

    def include(self, datum):
        self.n += 1
        self.delta = datum - self.mean
        self.mean += self.delta / self.n
        self.M2 += self.delta * (datum - self.mean)

    def variance(self):
        return self.M2 / (self.n - self.ddof)
    

In [10]:
from collections import defaultdict

domain_desc_stats_map = defaultdict(OnlineStats)

with open('TickerTick-stock-news-dataset.2023-11-23.json', "r") as f:
    pbar = tqdm()
    while (line := f.readline()):
        story = json.loads(line)
        if "url" in story:
            url = story["url"]
            domain = urlparse(url).hostname
            domain_desc_stats_map[domain].include(len(story["description"]) if "description" in story else 0)
        pbar.update(1)
    pbar.close()

len(domain_desc_stats_map)

9867339it [02:40, 61560.39it/s]


32479

In [11]:
domain_desc_stats = []

for domain, stats in domain_desc_stats_map.items():
    if stats.n > 1000:
        domain_desc_stats.append({"domain": domain, "n": stats.n, "mean": stats.mean, "std": stats.variance() ** 0.5})

desc_stats_df = pd.DataFrame(domain_desc_stats)
desc_stats_df = desc_stats_df.sort_values(by="n", ascending=False)
desc_stats_df

,domain,n,mean,std
30,www.tickerreport.com,782351,355.701036,20.414411
137,finance.yahoo.com,762210,234.421086,342.909783
35,seekingalpha.com,446532,0.030849,2.914709
29,www.etfdailynews.com,263621,354.313712,20.617613
28,www.reddit.com,229657,791.079379,1187.396293
...,...,...,...,...
652,news.cgtn.com,1023,5203.140762,1989.244992
176,www.gq.com,1019,95.922473,52.830705
618,www.datadoghq.com,1018,178.916503,270.155249
381,www.anandtech.com,1008,1510.179563,1832.775524


In [12]:
# Filter the original dataset to only include Yahoo Finance stories and save as CSV

stories = []
with open(f'YahooFinanceFiltered-{yf_news_data_file_path}', "w") as wf:
   with open(yf_news_data_file_path, "r") as rf:
      pbar = tqdm()
      while (line := rf.readline()):
         story = json.loads(line)
         url = story["url"]
         if urlparse(url).hostname == "finance.yahoo.com":
            stories.append(story)
         pbar.update(1)
      pbar.close()
   json.dump(stories, wf, indent=2)